https://github.com/rafaelnduarte/Spotify_K-Means_Clustering/blob/master/Retrieving_Data_from_Spotify.ipynb

In [1]:
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util
import sys

In [51]:
username = 'ray1d5u3xeb2e8c1nkpnwsnig'
client_id = 'bbf8d106196049c7831e658b73eb7a72'
client_secret = 'a249efe0cd3c411fa6182dfa2f4ce048'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [52]:
auth = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)

token = auth.get_access_token()
spotify = spotipy.Spotify(auth=token)
print(token)
print(spotify)

{'access_token': 'BQCt99CWsoJ343_F4zsAx37pVKdUmMcMlljZ-LmPC48GhEB264zWJVRAsNMgwR7rch_5XFQoFdDWiTb8ces', 'token_type': 'Bearer', 'expires_in': 3600, 'expires_at': 1648428351}


C:\Users\DAZZ\AppData\Local\Temp/ipykernel_7064/2508113278.py:6: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = auth.get_access_token()


In [45]:
token = token['access_token']

In [54]:
# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000,50):
    track_results = sp.search(q='year:2021', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

In [55]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 1000


In [56]:
# putting the data in a dataframe
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Carolina Gaitán - La Gaita,We Don't Talk About Bruno,52xJxFP6TqMuO4Yt0eOkMz,95
1,LYMEY,END OF YEAR 2021,6usFCFN0F69JcAaHtB55sg,28
2,Coldplay,My Universe,3FeVmId7tL5YN8B7R3imoM,92
3,Elvis Costello & The Attractions,Pump It Up - 2021 Remaster,3oyc1mIdCBGaU55wX7otqM,67
4,Kodak Black,Super Gremlin,4A8cWXxKfIL3lAyUDzXbCF,91


In [57]:
# info on the dataset
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  1000 non-null   object
 1   track_name   1000 non-null   object
 2   track_id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [58]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

36

In [59]:
# dropping duplicates
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [60]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [61]:
# dropping duplicates
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [62]:
# checking the shape of the dataframe
df_tracks.shape

(964, 4)

In [63]:
# timeit library to measure the time needed to run this code
import timeit
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.4899481000002197


In [64]:
# checking the results
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 964


In [65]:
# putting the data in a dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (964, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.577,0.450,0,-8.516,0,0.0834,0.35700,0.000000,0.111,0.830,205.863,audio_features,52xJxFP6TqMuO4Yt0eOkMz,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,https://api.spotify.com/v1/tracks/52xJxFP6TqMu...,https://api.spotify.com/v1/audio-analysis/52xJ...,216120,4
1,0.429,0.829,6,-5.071,0,0.0604,0.02380,0.000174,0.673,0.278,127.083,audio_features,6usFCFN0F69JcAaHtB55sg,spotify:track:6usFCFN0F69JcAaHtB55sg,https://api.spotify.com/v1/tracks/6usFCFN0F69J...,https://api.spotify.com/v1/audio-analysis/6usF...,1472769,4
2,0.588,0.701,9,-6.390,1,0.0402,0.00813,0.000000,0.200,0.443,104.988,audio_features,3FeVmId7tL5YN8B7R3imoM,spotify:track:3FeVmId7tL5YN8B7R3imoM,https://api.spotify.com/v1/tracks/3FeVmId7tL5Y...,https://api.spotify.com/v1/audio-analysis/3FeV...,228000,4
3,0.645,0.809,11,-6.120,1,0.0385,0.00921,0.001080,0.106,0.966,138.978,audio_features,3oyc1mIdCBGaU55wX7otqM,spotify:track:3oyc1mIdCBGaU55wX7otqM,https://api.spotify.com/v1/tracks/3oyc1mIdCBGa...,https://api.spotify.com/v1/audio-analysis/3oyc...,196680,4
4,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.357,0.106,72.993,audio_features,4A8cWXxKfIL3lAyUDzXbCF,spotify:track:4A8cWXxKfIL3lAyUDzXbCF,https://api.spotify.com/v1/tracks/4A8cWXxKfIL3...,https://api.spotify.com/v1/audio-analysis/4A8c...,200548,4


In [66]:
# info on the dataframe
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      964 non-null    float64
 1   energy            964 non-null    float64
 2   key               964 non-null    int64  
 3   loudness          964 non-null    float64
 4   mode              964 non-null    int64  
 5   speechiness       964 non-null    float64
 6   acousticness      964 non-null    float64
 7   instrumentalness  964 non-null    float64
 8   liveness          964 non-null    float64
 9   valence           964 non-null    float64
 10  tempo             964 non-null    float64
 11  type              964 non-null    object 
 12  id                964 non-null    object 
 13  uri               964 non-null    object 
 14  track_href        964 non-null    object 
 15  analysis_url      964 non-null    object 
 16  duration_ms       964 non-null    int64  
 1

In [67]:
# dropping unnecessary columns
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(964, 14)

In [68]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (964, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Carolina Gaitán - La Gaita,We Don't Talk About Bruno,52xJxFP6TqMuO4Yt0eOkMz,95,0.577,0.450,0,-8.516,0,0.0834,0.35700,0.000000,0.111,0.830,205.863,216120,4
1,LYMEY,END OF YEAR 2021,6usFCFN0F69JcAaHtB55sg,28,0.429,0.829,6,-5.071,0,0.0604,0.02380,0.000174,0.673,0.278,127.083,1472769,4
2,Coldplay,My Universe,3FeVmId7tL5YN8B7R3imoM,92,0.588,0.701,9,-6.390,1,0.0402,0.00813,0.000000,0.200,0.443,104.988,228000,4
3,Elvis Costello & The Attractions,Pump It Up - 2021 Remaster,3oyc1mIdCBGaU55wX7otqM,67,0.645,0.809,11,-6.120,1,0.0385,0.00921,0.001080,0.106,0.966,138.978,196680,4
4,Kodak Black,Super Gremlin,4A8cWXxKfIL3lAyUDzXbCF,91,0.825,0.414,2,-6.634,1,0.1440,0.00265,0.000000,0.357,0.106,72.993,200548,4


In [69]:
# info on the merge dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 0 to 963
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       964 non-null    object 
 1   track_name        964 non-null    object 
 2   track_id          964 non-null    object 
 3   popularity        964 non-null    int64  
 4   danceability      964 non-null    float64
 5   energy            964 non-null    float64
 6   key               964 non-null    int64  
 7   loudness          964 non-null    float64
 8   mode              964 non-null    int64  
 9   speechiness       964 non-null    float64
 10  acousticness      964 non-null    float64
 11  instrumentalness  964 non-null    float64
 12  liveness          964 non-null    float64
 13  valence           964 non-null    float64
 14  tempo             964 non-null    float64
 15  duration_ms       964 non-null    int64  
 16  time_signature    964 non-null    int64  
dt

In [70]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [71]:
# creating a csv file
df.to_csv('spotify2021_1000.csv')

### 2022

In [72]:
# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000,50):
    track_results = sp.search(q='year:2022', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

In [73]:
# putting the data in a dataframe
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Gunna,pushin P (feat. Young Thug),3XOalgusokruzA5ZBA2Qcb,93
1,Ish Kevin,My Year (2022),16TeLUubmneRAhwmRAMbjM,32
2,DJ Snake,U Are My High,4db4crB87tyZ9ExtEoxgVc,6
3,Years & Years,King,75flwWxBabCyZbLUo36Nvp,7
4,Dove Cameron,Boyfriend,59CfNbkERJ3NoTXDvoURjj,95


In [74]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

95

In [75]:
# dropping duplicates
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [76]:
# timeit library to measure the time needed to run this code
import timeit
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 1
Time to run this code (in seconds): 2.427036600000065


In [77]:
# putting the data in a dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (853, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.773,0.422,1,-4.572,0,0.1870,0.00783,0.00693,0.129,0.488,77.502,audio_features,3XOalgusokruzA5ZBA2Qcb,spotify:track:3XOalgusokruzA5ZBA2Qcb,https://api.spotify.com/v1/tracks/3XOalgusokru...,https://api.spotify.com/v1/audio-analysis/3XOa...,136267,1
1,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.00000,0.187,0.737,139.439,audio_features,16TeLUubmneRAhwmRAMbjM,spotify:track:16TeLUubmneRAhwmRAMbjM,https://api.spotify.com/v1/tracks/16TeLUubmneR...,https://api.spotify.com/v1/audio-analysis/16Te...,161808,3
2,0.503,0.783,0,-4.761,0,0.0385,0.02210,0.00223,0.140,0.213,125.122,audio_features,4db4crB87tyZ9ExtEoxgVc,spotify:track:4db4crB87tyZ9ExtEoxgVc,https://api.spotify.com/v1/tracks/4db4crB87tyZ...,https://api.spotify.com/v1/audio-analysis/4db4...,180480,4
3,0.559,0.837,4,-4.062,0,0.0391,0.06170,0.00000,0.393,0.446,119.958,audio_features,75flwWxBabCyZbLUo36Nvp,spotify:track:75flwWxBabCyZbLUo36Nvp,https://api.spotify.com/v1/tracks/75flwWxBabCy...,https://api.spotify.com/v1/audio-analysis/75fl...,214835,4
4,0.345,0.612,7,-6.543,0,0.0608,0.23200,0.00000,0.194,0.232,179.773,audio_features,59CfNbkERJ3NoTXDvoURjj,spotify:track:59CfNbkERJ3NoTXDvoURjj,https://api.spotify.com/v1/tracks/59CfNbkERJ3N...,https://api.spotify.com/v1/audio-analysis/59Cf...,153000,3


In [78]:
# info on the dataframe
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      853 non-null    float64
 1   energy            853 non-null    float64
 2   key               853 non-null    int64  
 3   loudness          853 non-null    float64
 4   mode              853 non-null    int64  
 5   speechiness       853 non-null    float64
 6   acousticness      853 non-null    float64
 7   instrumentalness  853 non-null    float64
 8   liveness          853 non-null    float64
 9   valence           853 non-null    float64
 10  tempo             853 non-null    float64
 11  type              853 non-null    object 
 12  id                853 non-null    object 
 13  uri               853 non-null    object 
 14  track_href        853 non-null    object 
 15  analysis_url      853 non-null    object 
 16  duration_ms       853 non-null    int64  
 1

In [79]:
# dropping unnecessary columns
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(853, 14)

In [80]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (853, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Gunna,pushin P (feat. Young Thug),3XOalgusokruzA5ZBA2Qcb,93,0.773,0.422,1,-4.572,0,0.1870,0.00783,0.00693,0.129,0.488,77.502,136267,1
1,Ish Kevin,My Year (2022),16TeLUubmneRAhwmRAMbjM,32,0.521,0.683,5,-6.240,0,0.5010,0.15900,0.00000,0.187,0.737,139.439,161808,3
2,DJ Snake,U Are My High,4db4crB87tyZ9ExtEoxgVc,6,0.503,0.783,0,-4.761,0,0.0385,0.02210,0.00223,0.140,0.213,125.122,180480,4
3,Years & Years,King,75flwWxBabCyZbLUo36Nvp,7,0.559,0.837,4,-4.062,0,0.0391,0.06170,0.00000,0.393,0.446,119.958,214835,4
4,Dove Cameron,Boyfriend,59CfNbkERJ3NoTXDvoURjj,95,0.345,0.612,7,-6.543,0,0.0608,0.23200,0.00000,0.194,0.232,179.773,153000,3


In [81]:
# info on the merge dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853 entries, 0 to 852
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       853 non-null    object 
 1   track_name        853 non-null    object 
 2   track_id          853 non-null    object 
 3   popularity        853 non-null    int64  
 4   danceability      853 non-null    float64
 5   energy            853 non-null    float64
 6   key               853 non-null    int64  
 7   loudness          853 non-null    float64
 8   mode              853 non-null    int64  
 9   speechiness       853 non-null    float64
 10  acousticness      853 non-null    float64
 11  instrumentalness  853 non-null    float64
 12  liveness          853 non-null    float64
 13  valence           853 non-null    float64
 14  tempo             853 non-null    float64
 15  duration_ms       853 non-null    int64  
 16  time_signature    853 non-null    int64  
dt

In [82]:
# creating a csv file
df.to_csv('spotify2022_1000.csv')